In [ ]:
pip install dlib

In [ ]:
pip install face_recognition

     |████████████████████████████████| 100.2MB 43kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=0e6a9b9dd22b224017df0e215cc862e1fe0bd125ff1703292e06ca0badcba9b3
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
pip install --upgrade imutils

Requirement already up-to-date: imutils in /usr/local/lib/python3.6/dist-packages (0.5.3)


In [ ]:
from scipy.spatial import distance as dist
from scipy import ndimage
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
from imutils import paths
from google.colab.patches import cv2_imshow
import face_recognition
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import os

In [ ]:
print(cv2.__version__
      )

4.1.2


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':'1Hi3g0FoBxzSZZGtB4KQLmACK5dbD_8nk'})
downloaded.GetContentFile('shape_predictor_68_face_landmarks.dat')

In [ ]:
import tarfile
tar = tarfile.open("5-celebrity-faces-dataset-new.tar.gz")
tar.extractall()
tar.close()

In [ ]:
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images('/content/5-celebrity-faces-dataset-new/data/train'))
knownEncodings = []
knownNames = []

[INFO] quantifying faces...


In [ ]:
for (i, imagePath) in enumerate(imagePaths):
  name = imagePath.split(os.path.sep)[-2]
  image = cv2.imread(imagePath)
  rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  boxes = face_recognition.face_locations(rgb, model="mmod_human_face_detector.dat")
  encodings = face_recognition.face_encodings(rgb, boxes)
  for encoding in encodings:
    knownEncodings.append(encoding)
    knownNames.append(name)
print("[INFO] serializing encodings...")
data = {"encodings": knownEncodings, "names": knownNames}

[INFO] serializing encodings...


In [ ]:
def eye_aspect_ratio(eye):
  A = dist.euclidean(eye[1], eye[5])
  B = dist.euclidean(eye[2], eye[4])
  C = dist.euclidean(eye[0], eye[3])
  ear = (A + B) / (2.0 * C)
  return ear

In [ ]:
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3
COUNTER = 0
TOTAL = 0

In [ ]:
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

[INFO] loading facial landmark predictor...


In [ ]:
print("[INFO] starting video stream thread...")
#vs = FileVideoStream('/content/VID_20200511_131132773.mp4').start()
vs = cv2.VideoCapture('/content/VID_20200511_131132773.mp4')
fileStream = True
#frame_width = int(vs.get(3)) vs.get(cv2.CAP_PROP_FRAME_HEIGHT)
#frame_height = int(vs.get(4))
#out = cv2.VideoWriter('output_vid.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (450,450))
#out = cv2.VideoWriter('output.avi', -1, 20.0, (1080.0,1920.0))
out = cv2.VideoWriter('output_.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 20.0, (1080,1920))

[INFO] starting video stream thread...


In [ ]:
h = vs.get(cv2.CAP_PROP_FRAME_HEIGHT)
w = vs.get(cv2.CAP_PROP_FRAME_WIDTH)
print(h)
print(w)

1080.0
1920.0


In [ ]:
face_cascPath = '/content/haarcascade_frontalface_alt.xml'
face_detector = cv2.CascadeClassifier(face_cascPath)

In [ ]:
while True:
  #if fileSeam and not vs.more():
    #break
  ret, frame = vs.read()
  if not ret:
    break
  (h, w) = frame.shape[:2]
  frame = imutils.resize(frame, width=450)
  #frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  rects = detector(gray, 0)
  faces = face_detector.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(50, 50),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
  for (i, rect) in enumerate(rects):
    (x,y,w,h) = faces[i]
    encoding = face_recognition.face_encodings(rgb, [(y, x+w, y+h, x)])[0]
    matches = face_recognition.compare_faces(data["encodings"], encoding)
    name = "Unknown"
    if True in matches:
      matchedIdxs = [i for (i, b) in enumerate(matches) if b]
      counts = {}
      for i in matchedIdxs:
        name = data["names"][i]
        counts[name] = counts.get(name, 0) + 1
      name = max(counts, key=counts.get)
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
    if ear < EYE_AR_THRESH:
        
    else:
      if COUNTER >= EYE_AR_CONSEC_FRAMES:
        TOTAL += 1
      COUNTER = 0
    cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    if TOTAL >= 3:
      cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
      y = y - 15 if y - 15 > 15 else y + 15
      cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
  out.write(frame)
  cv2_imshow(frame)
  key = cv2.waitKey(1) & 0xFF
  if key == ord("q"):
    break
#out.release()
vs.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from PIL import Image
import pytesseract
from pytesseract import Output
import re
import os
import math

In [ ]:
pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=1cb1ad17a661e8df0d25c30ec90eafed4da64fbb1c217983fb71f27ff992a743
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract


In [ ]:
for i in range(0,8):
  print(i)

0
1
2
3
4
5
6
7


In [ ]:
class Aadhaar_Card():
  def __init__(self,config = {'orient' : True,'skew' : True,'crop': True,'contrast' : True,'psm': [3,4,6],'mask_color': (0, 165, 255), 'brut_psm': [6]}):
    self.config = config
  def validate(self,aadhaarNum):
    mult = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 0, 6, 7, 8, 9, 5], [2, 3, 4, 0, 1, 7, 8, 9, 5, 6], [3, 4, 0, 1, 2, 8, 9, 5, 6, 7], [4, 0, 1, 2, 3, 9, 5, 6, 7, 8], [5, 9, 8, 7, 6, 0, 4, 3, 2, 1], [6, 5, 9, 8, 7, 1, 0, 4, 3, 2], [7, 6, 5, 9, 8, 2, 1, 0, 4, 3], [8, 7, 6, 5, 9, 3, 2, 1, 0, 4], [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]]
    perm = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 5, 7, 6, 2, 8, 3, 0, 9, 4], [5, 8, 0, 3, 7, 9, 6, 1, 4, 2], [8, 9, 1, 6, 0, 4, 3, 5, 2, 7], [9, 4, 5, 3, 1, 2, 6, 8, 7, 0], [4, 2, 8, 6, 5, 7, 3, 9, 0, 1], [2, 7, 9, 3, 8, 0, 6, 4, 1, 5], [7, 0, 4, 6, 9, 1, 3, 2, 5, 8]]
    try:
      i = len(aadhaarNum)
      j = 0
      x = 0
      while i > 0:
        i -= 1
        x = mult[x][perm[(j % 8)][int(aadhaarNum[i])]]
        j += 1
      if x == 0:
        return 1
      else:
        return 0
    except ValueError:
      return 0
    except IndexError:
      return 0
  def extract(self, path):
    self.image_path = path
    self.read_image_cv()
    if self.config['orient']:
      self.cv_img = self.rotate(self.cv_img)
    if self.config['skew']:
      print("skewness correction not available")
    if self.config['crop']:
      print("Smart Crop not available")
    if self.config['contrast']:
      self.cv_img  = self.contrast_image(self.cv_img)
      print("correcting contrast")
    aadhaars = set()
    for i in range(len(self.config['psm'])):
      t = self.text_extractor(self.cv_img,self.config['psm'][i])
      anum = self.is_aadhaar_card(t)
      uid = self.find_uid(t)
      if anum != "Not Found" and len(uid) == 0:
        if len(anum) - anum.count(' ') == 12:
          aadhaars.add(anum.replace(" ", ""))
      if anum == "Not Found" and len(uid) != 0:
        aadhaars.add(uid[0].replace(" ", ""))
      if anum != "Not Found" and len(uid) != 0:
        if len(anum) - anum.count(' ') == 12:
          aadhaars.add(anum.replace(" ", ""))
        aadhaars.add(uid[0].replace(" ", ""))
    return list(aadhaars)
  def mask_image(self, path, write, aadhaar_list):
    self.mask_count = 0
    self.mask = cv2.imread(str(path), cv2.IMREAD_COLOR)
    for j in range(len(self.config['psm'])):
      for i in range(len(aadhaar_list)):
        if (self.mask_aadhaar(aadhaar_list[i],write,self.config['psm'][j]))>0:
          self.mask_count = self.mask_count + 1
    cv2.imwrite(write,self.mask)
    return self.mask_count
  def mask_aadhaar(self, uid, out_path, psm):
    d = self.box_extractor(self.mask, psm)
    n_boxes = len(d['level'])
    color = self.config['mask_color']
    count_of_match = 0
    uid_8 = uid[:8]
    for i in range(n_boxes):
      string = d['text'][i].strip()
      if string.isdigit() and string in uid_8 and len(string)>=2:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        cv2.rectangle(self.mask, (x, y), (x + w, y + h), color, cv2.FILLED)
        count_of_match = count_of_match + 1
      else:
        count_of_match = count_of_match + 0
    return count_of_match
  def read_image_cv(self):
    self.cv_img = cv2.imread(str(self.image_path), cv2.IMREAD_COLOR)
  def mask_nums(self, input_file, output_file):
    img = cv2.imread(str(input_file), cv2.IMREAD_COLOR)
    j = 0
    for i in range(len(self.config['brut_psm'])):
      d = self.box_extractor(img,self.config['brut_psm'][i])
      n_boxes = len(d['level'])
      color = self.config['mask_color']
      for i in range(n_boxes):
        string = d['text'][i].strip()
        if string.isdigit() and len(string)==4 and j<2:
          j = j + 1
          (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
          cv2.rectangle(img, (x, y), (x + w, y + h), color, cv2.FILLED)
        else:
          j = 0
    cv2.imwrite(output_file,img)
    return "Done"
  def rotate_only(self, img, angle_in_degrees):
    self.img = img
    self.angle_in_degrees = angle_in_degrees
    rotated = ndimage.rotate(self.img, self.angle_in_degrees)
    return rotated
  def is_image_upside_down(self, img):
    self.img = img
    face_locations = face_recognition.face_locations(self.img)
    encodings = face_recognition.face_encodings(self.img, face_locations)
    image_is_upside_down = (len(encodings) == 0)
    return image_is_upside_down
  def rotate(self,img):
    self.img = img
    img_gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
    lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)
    img_copy = self.img.copy()
    for x in range(0, len(lines)):
      for x1,y1,x2,y2 in lines[x]:
        cv2.line(img_copy,(x1,y1),(x2,y2),(0,255,0),2)
    angles = []
    for x1, y1, x2, y2 in lines[0]:
      angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
      angles.append(angle)
    median_angle = np.median(angles)
    img_rotated = self.rotate_only(self.img, median_angle)
    if self.is_image_upside_down(img_rotated):
      print("rotate to 180 degree")
      angle = -180
      img_rotated_final = self.rotate_only(img_rotated, angle)
      if self.is_image_upside_down(img_rotated_final):
        print("Kindly check the uploaded image, face encodings still not found!")
        return img_rotated
      else:
        print("image is now straight")
        return img_rotated_final
    else:
      print("image is straight")
      return img_rotated
  def contrast_image(self, img):
    self.img = img
    gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    return thresh
  def text_extractor(self, img, psm):
    config  = ('-l eng --oem 3 --psm '+ str(psm))
    t = pytesseract.image_to_string(img, lang='eng', config = config)
    return t
  def box_extractor(self, img, psm):
    config  = ('-l eng --oem 3 --psm '+ str(psm))
    t = pytesseract.image_to_data(img, lang='eng', output_type=Output.DICT, config=config)
    return t
  def find_uid(self,text2):
    uid = set()
    try:
      newlist = []
      for xx in text2.split('\n'):
        newlist.append(xx)
      newlist = list(filter(lambda x: len(x) > 12, newlist))
      for no in newlist:
        if re.match("^[0-9 ]+$", no):
          uid.add(no)
    except Exception:
      pass
    return list(uid)
  def is_aadhaar_card(self, text):
    res=text.split()
    aadhaar_number=''
    for word in res:
      if len(word) == 4 and word.isdigit():
        aadhaar_number=aadhaar_number  + word + ' '
    if len(aadhaar_number)>=14:
      return aadhaar_number
    else:
      return "Not Found"

In [ ]:
config = {'orient' : True,   #corrects orientation of image default -> True
          'skew' : True,     #corrects skewness of image default -> True
          'crop': True,      #crops document out of image default -> True
          'contrast' : True, #Bnw for Better OCR default -> True
          'psm': [3,4,6],    #Google Tesseract psm modes default -> 3,4,6 
          'mask_color': (0, 165, 255),  #Masking color BGR Format
          'brut_psm': [6]    #Keep only one for brut mask (6) is good to start
          }

In [ ]:
objj = Aadhaar_Card(config)

In [ ]:
aadhaar_list = objj.extract("/content/WhatsApp Image 2020-01-15 at 12.43.13.jpeg")

rotate to 180 degree
Kindly check the uploaded image, face encodings still not found!
skewness correction not available
Smart Crop not available
correcting contrast


In [ ]:
aadhaar_list

['472894739406']

In [ ]:
aadhar_1 = aadhaar_list[0]

In [ ]:
A = aadhar_1[:8]
A

'47289473'

In [ ]:
flag = objj.mask_image("/content/WhatsApp Image 2020-01-15 at 12.43.13.jpeg", "result2.jpeg", aadhaar_list)

In [ ]:
flag2 = objj.mask_nums("/content/WhatsApp Image 2020-01-15 at 12.43.33.jpeg", "result1.jpeg")

In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 92kB 7.2MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 133kB 31.1MB/s 
     |████████████████████████████████| 28.2MB 116kB/s 
     |████████████████████████████████| 696kB 48.9MB/s 
     |████████████████████████████████| 307kB 48.6MB/s 
     |████████████████████████████████| 100.2MB 113kB/s 
  Created wheel for pytesseract: filename=pytesseract-0.3.3-py2.py3-none-any.whl size=13398 sha256=df1bd95853a22cc28e06baf16222f037d3aeba45577fbe1b4842bffa24886f5e
  Stored in directory: /root/.cache/pip/wheels/82/1a/d0/a884540a5602305f5bb7ff44f05550be20453edce8fae69241
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=0bc5acefe3892dc1941ed46405b2e8c9916d78a6ec398c7bd0ccccf1c0998453
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built pytesseract face-rec

In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,198 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
!pip install pytesseract

In [ ]:
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libleptonica-dev
The following NEW packages will be installed:
  libleptonica-dev libtesseract-dev
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 2,755 kB of archives.
After this operation, 13.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libleptonica-dev amd64 1.75.3-3 [1,308 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtesseract-dev amd64 4.00~git2288-10f4998a-2 [1,447 kB]
Fetched 2,755 kB in 1s (3,269 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This fr